In [1]:
import pandas as pd
import os
import csv
from pathlib import Path
import datetime as dt
from uszipcode import SearchEngine

## LOAD LOCATION DATA

In [2]:
zipcodes_coords_df = pd.read_csv("./resources/zipcodes_coordinates.csv")
zipcodes_coords_df

,RegionID,City,State,Zip Code,Latitude,Longitude
0,17426,Chicago,Illinois,60601,41.843935,-87.786352
1,18959,Las Vegas,Nevada,89101,36.167145,-115.139644
2,38128,Dallas,Texas,75201,32.785467,-96.798295
3,10920,Columbus,Ohio,43085,40.095385,-83.037600
4,12455,Louisville,Kentucky,40202,38.251228,-85.749400
...,...,...,...,...,...,...
721,26869,Royersford,Pennsylvania,19468,40.206897,-75.529549
722,36173,Miramar Beach,Florida,32550,30.384095,-86.349680
723,21021,Vail,Arizona,85641,31.955139,-110.762746
724,25643,Longboat Key,Florida,34228,27.389929,-82.639538


In [3]:
# Create list of viable regionID's
good_id_list = zipcodes_coords_df['RegionID'].tolist()

### Load cleaned (non-transposed) dataset, then remove rows without matching location data.

In [6]:
time_df = pd.read_csv("./Resources/Sale_Prices_City_sample.csv")
time_df

,RegionID,RegionName,StateName,SizeRank,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,...,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03
0,17426,Chicago,Illinois,4,325100.0,314800.0,286900.0,274600.0,268500.0,264400.0,...,271500.0,266500.0,264900.0,265000.0,264100.0,264300.0,270000.0,281400.0,302900.0,309200.0
1,18959,Las Vegas,Nevada,8,244400.0,240100.0,234600.0,227200.0,218100.0,206900.0,...,266000.0,268200.0,268400.0,270400.0,272200.0,274400.0,277500.0,278900.0,283000.0,284700.0
2,38128,Dallas,Texas,10,148400.0,142300.0,150900.0,157300.0,164100.0,163400.0,...,300600.0,301100.0,301500.0,310100.0,314600.0,316900.0,307100.0,321500.0,315500.0,321700.0
3,10920,Columbus,Ohio,19,116000.0,112100.0,114900.0,113800.0,111900.0,108200.0,...,152600.0,154800.0,158500.0,157700.0,162900.0,166600.0,172700.0,175300.0,173600.0,177600.0
4,12455,Louisville,Kentucky,20,124800.0,123500.0,122100.0,121900.0,121500.0,122100.0,...,172700.0,175900.0,176000.0,173900.0,175400.0,174600.0,176600.0,170800.0,173100.0,176200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,27950,Wildwood,New Jersey,3649,268300.0,256500.0,265200.0,289400.0,305100.0,294300.0,...,260100.0,260000.0,267200.0,264100.0,263300.0,266200.0,275300.0,274000.0,264500.0,270200.0
858,36173,Miramar Beach,Florida,3661,342200.0,362900.0,394800.0,434700.0,407400.0,364700.0,...,375700.0,384500.0,379300.0,374100.0,368200.0,370700.0,412500.0,408900.0,409000.0,379500.0
859,21021,Vail,Arizona,3681,280700.0,281200.0,267000.0,258700.0,256600.0,253200.0,...,277600.0,277800.0,283100.0,286700.0,297000.0,297200.0,292100.0,275100.0,265400.0,269300.0
860,25643,Longboat Key,Florida,3690,646800.0,613000.0,546400.0,602800.0,643800.0,696400.0,...,567400.0,554700.0,502000.0,495600.0,535300.0,557900.0,577100.0,557900.0,565600.0,579700.0


In [7]:
# remove rows from time_df that aren't in the list of good locations
for index, row in time_df.iterrows():
    if row['RegionID'] not in good_id_list:
        time_df.drop(index, inplace=True)
        
time_df

,RegionID,RegionName,StateName,SizeRank,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,...,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03
0,17426,Chicago,Illinois,4,325100.0,314800.0,286900.0,274600.0,268500.0,264400.0,...,271500.0,266500.0,264900.0,265000.0,264100.0,264300.0,270000.0,281400.0,302900.0,309200.0
1,18959,Las Vegas,Nevada,8,244400.0,240100.0,234600.0,227200.0,218100.0,206900.0,...,266000.0,268200.0,268400.0,270400.0,272200.0,274400.0,277500.0,278900.0,283000.0,284700.0
2,38128,Dallas,Texas,10,148400.0,142300.0,150900.0,157300.0,164100.0,163400.0,...,300600.0,301100.0,301500.0,310100.0,314600.0,316900.0,307100.0,321500.0,315500.0,321700.0
3,10920,Columbus,Ohio,19,116000.0,112100.0,114900.0,113800.0,111900.0,108200.0,...,152600.0,154800.0,158500.0,157700.0,162900.0,166600.0,172700.0,175300.0,173600.0,177600.0
4,12455,Louisville,Kentucky,20,124800.0,123500.0,122100.0,121900.0,121500.0,122100.0,...,172700.0,175900.0,176000.0,173900.0,175400.0,174600.0,176600.0,170800.0,173100.0,176200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
856,26869,Royersford,Pennsylvania,3641,242600.0,229700.0,224500.0,217700.0,217700.0,215200.0,...,284500.0,272100.0,275000.0,273900.0,262800.0,264100.0,265200.0,272300.0,248800.0,235100.0
858,36173,Miramar Beach,Florida,3661,342200.0,362900.0,394800.0,434700.0,407400.0,364700.0,...,375700.0,384500.0,379300.0,374100.0,368200.0,370700.0,412500.0,408900.0,409000.0,379500.0
859,21021,Vail,Arizona,3681,280700.0,281200.0,267000.0,258700.0,256600.0,253200.0,...,277600.0,277800.0,283100.0,286700.0,297000.0,297200.0,292100.0,275100.0,265400.0,269300.0
860,25643,Longboat Key,Florida,3690,646800.0,613000.0,546400.0,602800.0,643800.0,696400.0,...,567400.0,554700.0,502000.0,495600.0,535300.0,557900.0,577100.0,557900.0,565600.0,579700.0


In [8]:
# recreate transposed dataset
time_df = time_df.drop(columns=["RegionName", "StateName", "SizeRank"])
time_df = time_df.astype({'RegionID': 'str'})
time_transposed = time_df.transpose()
time_transposed.reset_index(drop = False, inplace = True)
time_transposed = time_transposed.rename(columns=time_transposed.iloc[0])
time_transposed = time_transposed.drop([0, 0])
time_transposed = time_transposed.rename(columns={"RegionID": 'DATE'})
time_transposed['DATE'] = pd.to_datetime(time_transposed['DATE'], infer_datetime_format=True)
time_transposed

/var/folders/gm/c_xwwqf15t71b9pr19cpd2yw0000gn/T/ipykernel_18737/40715547.py:9: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  time_transposed['DATE'] = pd.to_datetime(time_transposed['DATE'], infer_datetime_format=True)


,DATE,17426,18959,38128,10920,12455,13121,11093,41568,13373,...,18759,396880,21446,24162,15959,26869,36173,21021,25643,48710
1,2008-03-01,325100.0,244400.0,148400.0,116000.0,124800.0,222600.0,185800.0,382500.0,293000.0,...,181400.0,288500.0,101300.0,306500.0,580700.0,242600.0,342200.0,280700.0,646800.0,360700.0
2,2008-04-01,314800.0,240100.0,142300.0,112100.0,123500.0,214500.0,179800.0,380900.0,284600.0,...,188400.0,314900.0,105700.0,304400.0,557600.0,229700.0,362900.0,281200.0,613000.0,342700.0
3,2008-05-01,286900.0,234600.0,150900.0,114900.0,122100.0,201600.0,172100.0,382100.0,282900.0,...,199800.0,320000.0,108700.0,281200.0,591700.0,224500.0,394800.0,267000.0,546400.0,316500.0
4,2008-06-01,274600.0,227200.0,157300.0,113800.0,121900.0,194800.0,163500.0,382400.0,284700.0,...,202500.0,343100.0,122900.0,271200.0,652800.0,217700.0,434700.0,258700.0,602800.0,332800.0
5,2008-07-01,268500.0,218100.0,164100.0,111900.0,121500.0,190600.0,163700.0,386800.0,287000.0,...,199400.0,317300.0,130000.0,252700.0,668800.0,217700.0,407400.0,256600.0,643800.0,332300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,2019-11-01,264300.0,274400.0,316900.0,166600.0,174600.0,247400.0,445400.0,566400.0,432200.0,...,375800.0,292800.0,193100.0,309100.0,808000.0,264100.0,370700.0,297200.0,557900.0,279400.0
142,2019-12-01,270000.0,277500.0,307100.0,172700.0,176600.0,251100.0,435500.0,577600.0,431900.0,...,376300.0,267400.0,205200.0,320800.0,839900.0,265200.0,412500.0,292100.0,577100.0,287100.0
143,2020-01-01,281400.0,278900.0,321500.0,175300.0,170800.0,254800.0,435100.0,580100.0,435400.0,...,369000.0,284300.0,219700.0,338400.0,736900.0,272300.0,408900.0,275100.0,557900.0,305300.0
144,2020-02-01,302900.0,283000.0,315500.0,173600.0,173100.0,263500.0,436900.0,575700.0,435200.0,...,364200.0,323700.0,219400.0,342600.0,682600.0,248800.0,409000.0,265400.0,565600.0,325600.0


### LOAD/CLEAN INFLATION DATA

In [13]:
file_path = Path("./Resources/CPIAUCNS.csv")
cpi_df = pd.read_csv(file_path)
cpi_df['DATE'] = pd.to_datetime(cpi_df['observation_date'], infer_datetime_format=True)
print(cpi_df.dtypes)
cpi_df

observation_date            object
CPIAUCNS                   float64
DATE                datetime64[ns]
dtype: object


/var/folders/gm/c_xwwqf15t71b9pr19cpd2yw0000gn/T/ipykernel_18737/231595496.py:3: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  cpi_df['DATE'] = pd.to_datetime(cpi_df['observation_date'], infer_datetime_format=True)


,observation_date,CPIAUCNS,DATE
0,1913-01-01,9.800,1913-01-01
1,1913-02-01,9.800,1913-02-01
2,1913-03-01,9.800,1913-03-01
3,1913-04-01,9.800,1913-04-01
4,1913-05-01,9.700,1913-05-01
...,...,...,...
1340,2024-09-01,315.301,2024-09-01
1341,2024-10-01,315.664,2024-10-01
1342,2024-11-01,315.493,2024-11-01
1343,2024-12-01,315.605,2024-12-01


In [14]:
# Merge inflation data into transposed dataset
time_transposed = time_transposed.merge(cpi_df, how='inner', on='DATE')
inf = time_transposed['CPIAUCNS']
time_transposed = time_transposed.drop(columns=['CPIAUCNS'])
time_transposed.insert(loc=1, column='INFLATION', value=inf)
time_transposed = time_transposed.dropna()
time_transposed

,DATE,INFLATION,17426,18959,38128,10920,12455,13121,11093,41568,...,396880,21446,24162,15959,26869,36173,21021,25643,48710,observation_date
0,2008-03-01,213.528,325100.0,244400.0,148400.0,116000.0,124800.0,222600.0,185800.0,382500.0,...,288500.0,101300.0,306500.0,580700.0,242600.0,342200.0,280700.0,646800.0,360700.0,2008-03-01
1,2008-04-01,214.823,314800.0,240100.0,142300.0,112100.0,123500.0,214500.0,179800.0,380900.0,...,314900.0,105700.0,304400.0,557600.0,229700.0,362900.0,281200.0,613000.0,342700.0,2008-04-01
2,2008-05-01,216.632,286900.0,234600.0,150900.0,114900.0,122100.0,201600.0,172100.0,382100.0,...,320000.0,108700.0,281200.0,591700.0,224500.0,394800.0,267000.0,546400.0,316500.0,2008-05-01
3,2008-06-01,218.815,274600.0,227200.0,157300.0,113800.0,121900.0,194800.0,163500.0,382400.0,...,343100.0,122900.0,271200.0,652800.0,217700.0,434700.0,258700.0,602800.0,332800.0,2008-06-01
4,2008-07-01,219.964,268500.0,218100.0,164100.0,111900.0,121500.0,190600.0,163700.0,386800.0,...,317300.0,130000.0,252700.0,668800.0,217700.0,407400.0,256600.0,643800.0,332300.0,2008-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,2019-11-01,257.208,264300.0,274400.0,316900.0,166600.0,174600.0,247400.0,445400.0,566400.0,...,292800.0,193100.0,309100.0,808000.0,264100.0,370700.0,297200.0,557900.0,279400.0,2019-11-01
141,2019-12-01,256.974,270000.0,277500.0,307100.0,172700.0,176600.0,251100.0,435500.0,577600.0,...,267400.0,205200.0,320800.0,839900.0,265200.0,412500.0,292100.0,577100.0,287100.0,2019-12-01
142,2020-01-01,257.971,281400.0,278900.0,321500.0,175300.0,170800.0,254800.0,435100.0,580100.0,...,284300.0,219700.0,338400.0,736900.0,272300.0,408900.0,275100.0,557900.0,305300.0,2020-01-01
143,2020-02-01,258.678,302900.0,283000.0,315500.0,173600.0,173100.0,263500.0,436900.0,575700.0,...,323700.0,219400.0,342600.0,682600.0,248800.0,409000.0,265400.0,565600.0,325600.0,2020-02-01


### LOAD/CLEAN INTEREST RATE DATA

In [34]:
interest_file = Path("./resources/FEDFUNDS.csv")
interest_df = pd.read_csv(interest_file)
interest_df['DATE'] = pd.to_datetime(interest_df['DATE'])
print(interest_df.dtypes)
interest_df

DATE        datetime64[ns]
FEDFUNDS           float64
dtype: object


,DATE,FEDFUNDS
0,1954-07-01,0.80
1,1954-08-01,1.22
2,1954-09-01,1.07
3,1954-10-01,0.85
4,1954-11-01,0.83
...,...,...
807,2021-10-01,0.08
808,2021-11-01,0.08
809,2021-12-01,0.08
810,2022-01-01,0.08


In [ ]:
# merge interest data into transposed dataset
time_transposed = time_transposed.merge(interest_df, how='inner', on='DATE')
interest = time_transposed['FEDFUNDS']
time_transposed = time_transposed.drop(columns=['FEDFUNDS'])
time_transposed.insert(loc=1, column='INTEREST_RATES', value=interest)
time_transposed = time_transposed.dropna()

In [37]:
time_transposed

,DATE,INTEREST_RATES,INFLATION,17426,18959,38128,10920,12455,13121,11093,...,21446,24162,15959,26869,36173,21021,25643,48710,observation_date,Average\nContract\nRate
0,2008-03-01,2.61,213.528,325100.0,244400.0,148400.0,116000.0,124800.0,222600.0,185800.0,...,101300.0,306500.0,580700.0,242600.0,342200.0,280700.0,646800.0,360700.0,2008-03-01,6.3
1,2008-04-01,2.28,214.823,314800.0,240100.0,142300.0,112100.0,123500.0,214500.0,179800.0,...,105700.0,304400.0,557600.0,229700.0,362900.0,281200.0,613000.0,342700.0,2008-04-01,6.2
2,2008-05-01,1.98,216.632,286900.0,234600.0,150900.0,114900.0,122100.0,201600.0,172100.0,...,108700.0,281200.0,591700.0,224500.0,394800.0,267000.0,546400.0,316500.0,2008-05-01,6.2
3,2008-06-01,2.00,218.815,274600.0,227200.0,157300.0,113800.0,121900.0,194800.0,163500.0,...,122900.0,271200.0,652800.0,217700.0,434700.0,258700.0,602800.0,332800.0,2008-06-01,6.3
4,2008-07-01,2.01,219.964,268500.0,218100.0,164100.0,111900.0,121500.0,190600.0,163700.0,...,130000.0,252700.0,668800.0,217700.0,407400.0,256600.0,643800.0,332300.0,2008-07-01,6.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,2019-11-01,1.55,257.208,264300.0,274400.0,316900.0,166600.0,174600.0,247400.0,445400.0,...,193100.0,309100.0,808000.0,264100.0,370700.0,297200.0,557900.0,279400.0,2019-11-01,4.1
141,2019-12-01,1.55,256.974,270000.0,277500.0,307100.0,172700.0,176600.0,251100.0,435500.0,...,205200.0,320800.0,839900.0,265200.0,412500.0,292100.0,577100.0,287100.0,2019-12-01,4.1
142,2020-01-01,1.55,257.971,281400.0,278900.0,321500.0,175300.0,170800.0,254800.0,435100.0,...,219700.0,338400.0,736900.0,272300.0,408900.0,275100.0,557900.0,305300.0,2020-01-01,4.1
143,2020-02-01,1.58,258.678,302900.0,283000.0,315500.0,173600.0,173100.0,263500.0,436900.0,...,219400.0,342600.0,682600.0,248800.0,409000.0,265400.0,565600.0,325600.0,2020-02-01,4.0


### LOAD/CLEAN MORTGAGE RATE DATA

In [38]:
mortgage_file = Path("./Resources/Mortgage_data_trimmed.csv")
mortgage_df = pd.read_csv(mortgage_file)[2:]

In [39]:
print(mortgage_df.columns)


Index(['Month', 'Average\nContract\nRate'], dtype='object')


In [41]:
mortgage_df = mortgage_df.rename(columns={'Month': 'DATE', 'Average\nContract\nRate': 'AVG_MORTGAGE_RATE'})
mortgage_df['DATE'] = mortgage_df['DATE'].astype(str).str.strip()
mortgage_df['DATE'] = pd.to_datetime(mortgage_df['DATE'], errors='coerce')
print(mortgage_df.dtypes)
mortgage_df

DATE                 datetime64[ns]
AVG_MORTGAGE_RATE           float64
dtype: object


,DATE,AVG_MORTGAGE_RATE
2,1998-01-01,7.3
3,1998-02-01,7.2
4,1998-03-01,7.2
5,1998-04-01,7.2
6,1998-05-01,7.3
...,...,...
276,2020-11-01,3.0
277,2020-12-01,3.0
278,2021-01-01,3.0
279,2021-02-01,3.0


In [42]:
time_transposed = time_transposed.merge(mortgage_df, how='inner', on='DATE')
mort = time_transposed['AVG_MORTGAGE_RATE']
time_transposed = time_transposed.drop(columns=['AVG_MORTGAGE_RATE'])
time_transposed.insert(loc=1, column='AVG_MORTGAGE_RATE', value=mort)
time_transposed = time_transposed.dropna()
time_transposed

,DATE,AVG_MORTGAGE_RATE,INTEREST_RATES,INFLATION,17426,18959,38128,10920,12455,13121,...,21446,24162,15959,26869,36173,21021,25643,48710,observation_date,Average\nContract\nRate
0,2008-03-01,6.3,2.61,213.528,325100.0,244400.0,148400.0,116000.0,124800.0,222600.0,...,101300.0,306500.0,580700.0,242600.0,342200.0,280700.0,646800.0,360700.0,2008-03-01,6.3
1,2008-04-01,6.2,2.28,214.823,314800.0,240100.0,142300.0,112100.0,123500.0,214500.0,...,105700.0,304400.0,557600.0,229700.0,362900.0,281200.0,613000.0,342700.0,2008-04-01,6.2
2,2008-05-01,6.2,1.98,216.632,286900.0,234600.0,150900.0,114900.0,122100.0,201600.0,...,108700.0,281200.0,591700.0,224500.0,394800.0,267000.0,546400.0,316500.0,2008-05-01,6.2
3,2008-06-01,6.3,2.00,218.815,274600.0,227200.0,157300.0,113800.0,121900.0,194800.0,...,122900.0,271200.0,652800.0,217700.0,434700.0,258700.0,602800.0,332800.0,2008-06-01,6.3
4,2008-07-01,6.5,2.01,219.964,268500.0,218100.0,164100.0,111900.0,121500.0,190600.0,...,130000.0,252700.0,668800.0,217700.0,407400.0,256600.0,643800.0,332300.0,2008-07-01,6.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,2019-11-01,4.1,1.55,257.208,264300.0,274400.0,316900.0,166600.0,174600.0,247400.0,...,193100.0,309100.0,808000.0,264100.0,370700.0,297200.0,557900.0,279400.0,2019-11-01,4.1
141,2019-12-01,4.1,1.55,256.974,270000.0,277500.0,307100.0,172700.0,176600.0,251100.0,...,205200.0,320800.0,839900.0,265200.0,412500.0,292100.0,577100.0,287100.0,2019-12-01,4.1
142,2020-01-01,4.1,1.55,257.971,281400.0,278900.0,321500.0,175300.0,170800.0,254800.0,...,219700.0,338400.0,736900.0,272300.0,408900.0,275100.0,557900.0,305300.0,2020-01-01,4.1
143,2020-02-01,4.0,1.58,258.678,302900.0,283000.0,315500.0,173600.0,173100.0,263500.0,...,219400.0,342600.0,682600.0,248800.0,409000.0,265400.0,565600.0,325600.0,2020-02-01,4.0


### LOAD/CLEAN UNEMPLOYMENT RATE DATA

In [44]:
unemployment_file = Path("./Resources/unemployment_data_usa_only.csv")
unemployment_df = pd.read_csv(unemployment_file)
unemployment_df['TIME'] = pd.to_datetime(unemployment_df['TIME'], errors='coerce')
unemployment_df = unemployment_df.rename(columns={'TIME': 'DATE', 'Value': 'UNEMPLOYMENT_RATE'})
print(unemployment_df.dtypes)
unemployment_df

DATE                 datetime64[ns]
UNEMPLOYMENT_RATE           float64
dtype: object


,DATE,UNEMPLOYMENT_RATE
0,2010-11-01,9.8
1,2010-12-01,9.3
2,2011-01-01,9.1
3,2011-02-01,9.0
4,2011-03-01,9.0
...,...,...
131,2021-10-01,4.6
132,2021-11-01,4.2
133,2021-12-01,3.9
134,2022-01-01,4.0


In [45]:
# merge unemployment rate data into transposed dataset
time_transposed = time_transposed.merge(unemployment_df, how='inner', on='DATE')
unemp = time_transposed['UNEMPLOYMENT_RATE']
time_transposed = time_transposed.drop(columns=['UNEMPLOYMENT_RATE'])
time_transposed.insert(loc=1, column='UNEMPLOYMENT_RATE', value=unemp)
time_transposed

,DATE,UNEMPLOYMENT_RATE,AVG_MORTGAGE_RATE,INTEREST_RATES,INFLATION,17426,18959,38128,10920,12455,...,21446,24162,15959,26869,36173,21021,25643,48710,observation_date,Average\nContract\nRate
0,2010-11-01,9.8,4.4,0.19,218.803,190000.0,117400.0,169400.0,84700.0,120800.0,...,147100.0,307700.0,584800.0,226800.0,275200.0,201500.0,437400.0,245900.0,2010-11-01,4.4
1,2010-12-01,9.3,4.5,0.18,219.179,190400.0,116600.0,169500.0,90600.0,123800.0,...,157500.0,298600.0,681400.0,230800.0,276300.0,213500.0,461800.0,261500.0,2010-12-01,4.5
2,2011-01-01,9.1,4.8,0.17,220.223,188500.0,115600.0,191900.0,89100.0,125900.0,...,138900.0,283000.0,607500.0,229700.0,284400.0,204100.0,432300.0,266300.0,2011-01-01,4.8
3,2011-02-01,9.0,4.9,0.16,221.309,189400.0,114000.0,184500.0,86000.0,124000.0,...,142900.0,279500.0,631500.0,216700.0,258100.0,194000.0,476900.0,244200.0,2011-02-01,4.9
4,2011-03-01,9.0,5.0,0.14,223.467,186300.0,110800.0,174500.0,79800.0,118300.0,...,136400.0,296800.0,541800.0,199800.0,253800.0,192400.0,440900.0,217800.0,2011-03-01,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,2019-11-01,3.6,4.1,1.55,257.208,264300.0,274400.0,316900.0,166600.0,174600.0,...,193100.0,309100.0,808000.0,264100.0,370700.0,297200.0,557900.0,279400.0,2019-11-01,4.1
109,2019-12-01,3.6,4.1,1.55,256.974,270000.0,277500.0,307100.0,172700.0,176600.0,...,205200.0,320800.0,839900.0,265200.0,412500.0,292100.0,577100.0,287100.0,2019-12-01,4.1
110,2020-01-01,3.5,4.1,1.55,257.971,281400.0,278900.0,321500.0,175300.0,170800.0,...,219700.0,338400.0,736900.0,272300.0,408900.0,275100.0,557900.0,305300.0,2020-01-01,4.1
111,2020-02-01,3.5,4.0,1.58,258.678,302900.0,283000.0,315500.0,173600.0,173100.0,...,219400.0,342600.0,682600.0,248800.0,409000.0,265400.0,565600.0,325600.0,2020-02-01,4.0


### LOAD/CLEAN GAS PRICE DATA

In [47]:
gas_file = Path('./Resources/gas_price.csv')
gas_df = pd.read_csv(gas_file)
gas_df['DATE'] = pd.to_datetime(gas_df['DATE'], errors='coerce')
print(gas_df.dtypes)
gas_df

DATE                datetime64[ns]
PRICE_PER_GALLON           float64
dtype: object


,DATE,PRICE_PER_GALLON
0,1993-04-01,1.078
1,1993-05-01,1.100
2,1993-06-01,1.097
3,1993-07-01,1.078
4,1993-08-01,1.062
...,...,...
342,2021-10-01,3.384
343,2021-11-01,3.491
344,2021-12-01,3.406
345,2022-01-01,3.413


In [48]:
# add gas price data to our master datasource
time_transposed = time_transposed.merge(gas_df, how='inner', on='DATE')
gas = time_transposed['PRICE_PER_GALLON']
time_transposed = time_transposed.drop(columns=['PRICE_PER_GALLON'])
time_transposed.insert(loc=1, column='PRICE_PER_GALLON', value=gas)
time_transposed = time_transposed.dropna()
time_transposed

,DATE,PRICE_PER_GALLON,UNEMPLOYMENT_RATE,AVG_MORTGAGE_RATE,INTEREST_RATES,INFLATION,17426,18959,38128,10920,...,21446,24162,15959,26869,36173,21021,25643,48710,observation_date,Average\nContract\nRate
0,2010-11-01,2.913,9.8,4.4,0.19,218.803,190000.0,117400.0,169400.0,84700.0,...,147100.0,307700.0,584800.0,226800.0,275200.0,201500.0,437400.0,245900.0,2010-11-01,4.4
1,2010-12-01,3.048,9.3,4.5,0.18,219.179,190400.0,116600.0,169500.0,90600.0,...,157500.0,298600.0,681400.0,230800.0,276300.0,213500.0,461800.0,261500.0,2010-12-01,4.5
2,2011-01-01,3.148,9.1,4.8,0.17,220.223,188500.0,115600.0,191900.0,89100.0,...,138900.0,283000.0,607500.0,229700.0,284400.0,204100.0,432300.0,266300.0,2011-01-01,4.8
3,2011-02-01,3.264,9.0,4.9,0.16,221.309,189400.0,114000.0,184500.0,86000.0,...,142900.0,279500.0,631500.0,216700.0,258100.0,194000.0,476900.0,244200.0,2011-02-01,4.9
4,2011-03-01,3.615,9.0,5.0,0.14,223.467,186300.0,110800.0,174500.0,79800.0,...,136400.0,296800.0,541800.0,199800.0,253800.0,192400.0,440900.0,217800.0,2011-03-01,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,2019-11-01,2.693,3.6,4.1,1.55,257.208,264300.0,274400.0,316900.0,166600.0,...,193100.0,309100.0,808000.0,264100.0,370700.0,297200.0,557900.0,279400.0,2019-11-01,4.1
109,2019-12-01,2.645,3.6,4.1,1.55,256.974,270000.0,277500.0,307100.0,172700.0,...,205200.0,320800.0,839900.0,265200.0,412500.0,292100.0,577100.0,287100.0,2019-12-01,4.1
110,2020-01-01,2.636,3.5,4.1,1.55,257.971,281400.0,278900.0,321500.0,175300.0,...,219700.0,338400.0,736900.0,272300.0,408900.0,275100.0,557900.0,305300.0,2020-01-01,4.1
111,2020-02-01,2.533,3.5,4.0,1.58,258.678,302900.0,283000.0,315500.0,173600.0,...,219400.0,342600.0,682600.0,248800.0,409000.0,265400.0,565600.0,325600.0,2020-02-01,4.0


In [49]:
# Create CSV with fully cleaned and incorporated data
time_transposed.to_csv('./Resources/full_data_set.csv', index=False)